In [3]:
%pip3 install numpy --user
# %pip install nltk
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
print('Setup done!')

UsageError: Line magic function `%pip3` not found.


In [ ]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('./input/intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',', ':', ';', '"', '\'']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.append(word_list)
        documents.append((word_list, intent['topic']))
        if intent['topic'] not in classes:
            classes.append(intent['topic'])

documents